In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
pd.options.display.float_format = '{:.2f}'.format

In [2]:
# Read in New York Times online csv file with daily COVID data
# Convert FIPS number to integer, convert date to datetime, save file to local csv

us_covid_df = pd.read_csv('https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv')
us_covid_df['fips'] = us_covid_df['fips'].fillna(0).astype(np.int64)
us_covid_df['date'] = pd.to_datetime(us_covid_df['date'], yearfirst=True)
us_covid_df.to_csv('us_covid.csv', index=False)

In [3]:
# Read in national population data provided by state, county, FIPS

us_pops_df = pd.read_csv('US_Populations.csv', engine='python', thousands=',')
us_pops_df

,FIPStxt,State,State2,Area_Name,Region,population
0,2000,Alaska,AK,Alaska,State,731545
1,2013,Alaska,AK,Aleutians East Borough,County,3337
2,2016,Alaska,AK,Aleutians West Census Area,County,5634
3,2020,Alaska,AK,Anchorage,County,288000
4,2050,Alaska,AK,Bethel Census Area,County,18386
...,...,...,...,...,...,...
3146,22119,Louisiana,LA,Webster,County,38340
3147,22121,Louisiana,LA,West Baton Rouge,County,26465
3148,22123,Louisiana,LA,West Carroll,County,10830
3149,22125,Louisiana,LA,West Feliciana,County,15568


In [4]:
# Merge New York Times data with population data

us_data_df = pd.merge(us_covid_df, us_pops_df, left_on=['county', 'state'], right_on=['Area_Name', 'State'])
us_data_df

,date,county,state,fips,cases,deaths,FIPStxt,State,State2,Area_Name,Region,population
0,2020-01-21,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
1,2020-01-22,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
2,2020-01-23,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
3,2020-01-24,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
4,2020-01-25,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
...,...,...,...,...,...,...,...,...,...,...,...,...
1202937,2021-04-21,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
1202938,2021-04-22,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
1202939,2021-04-23,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
1202940,2021-04-24,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86


In [5]:
# Separate out data from the state level

states = us_data_df[(us_data_df['Region'] == 'State')]
states

,date,county,state,fips,cases,deaths,FIPStxt,State,State2,Area_Name,Region,population
32183,2020-03-07,District of Columbia,District of Columbia,11001,1,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
32185,2020-03-08,District of Columbia,District of Columbia,11001,1,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
32187,2020-03-09,District of Columbia,District of Columbia,11001,4,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
32189,2020-03-10,District of Columbia,District of Columbia,11001,4,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
32191,2020-03-11,District of Columbia,District of Columbia,11001,10,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
...,...,...,...,...,...,...,...,...,...,...,...,...
740776,2021-04-21,Arkansas,Arkansas,5001,2088,36.00,5000,Arkansas,AR,Arkansas,State,3017804
740778,2021-04-22,Arkansas,Arkansas,5001,2089,36.00,5000,Arkansas,AR,Arkansas,State,3017804
740780,2021-04-23,Arkansas,Arkansas,5001,2089,36.00,5000,Arkansas,AR,Arkansas,State,3017804
740782,2021-04-24,Arkansas,Arkansas,5001,2092,36.00,5000,Arkansas,AR,Arkansas,State,3017804


In [6]:
# Separate out data specific to county level, drop redundant columns
#  Create infection and death rates, updating for each date in the dataframe

counties = us_data_df.drop(['fips', 'State','Area_Name'], axis=1)
counties['current_crate'] = counties['cases']*100000/counties['population']
counties['current_drate'] = counties['deaths']*100000/counties['population']
counties = counties[(counties['Region'] == 'County')]
counties

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate
0,2020-01-21,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
1,2020-01-22,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
2,2020-01-23,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
3,2020-01-24,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
4,2020-01-25,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
...,...,...,...,...,...,...,...,...,...,...,...
1202937,2021-04-21,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
1202938,2021-04-22,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
1202939,2021-04-23,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
1202940,2021-04-24,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00


In [7]:
# Isolate data from most current day.

counties_us = counties[(counties['date'] == '2021-04-25') & (counties['Region'] == 'County')]
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate
460,2021-04-25,Snohomish,Washington,35132,563.00,53061,WA,County,822083,4273.53,68.48
918,2021-04-25,Cook,Illinois,529165,10449.00,17031,IL,County,5150233,10274.58,202.88
1375,2021-04-25,Orange,California,269633,4933.00,6059,CA,County,3175692,8490.53,155.34
1831,2021-04-25,Maricopa,Arizona,534430,9836.00,4013,AZ,County,4485414,11914.84,219.29
2287,2021-04-25,Los Angeles,California,1232195,23802.00,6037,CA,County,10039107,12273.95,237.09
...,...,...,...,...,...,...,...,...,...,...,...
1202285,2021-04-25,Wheeler,Oregon,26,1.00,41069,OR,County,1332,1951.95,75.08
1202480,2021-04-25,King,Texas,11,0.00,48269,TX,County,272,4044.12,0.00
1202644,2021-04-25,Esmeralda,Nevada,38,0.00,32009,NV,County,873,4352.81,0.00
1202804,2021-04-25,Loving,Texas,1,0.00,48301,TX,County,169,591.72,0.00


In [8]:
# Create up to date rates so rankings can be performed using the latest data.

counties_us['drate'] = counties_us['deaths']/(counties_us['population']/100000)
counties_us['crate'] = counties_us['cases']/(counties_us['population']/100000)
counties_us = counties_us[(counties_us['Region'] == 'County')]
counties_us

<ipython-input-8-608f6ade14d5>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_us['drate'] = counties_us['deaths']/(counties_us['population']/100000)
<ipython-input-8-608f6ade14d5>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_us['crate'] = counties_us['cases']/(counties_us['population']/100000)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate
460,2021-04-25,Snohomish,Washington,35132,563.00,53061,WA,County,822083,4273.53,68.48,68.48,4273.53
918,2021-04-25,Cook,Illinois,529165,10449.00,17031,IL,County,5150233,10274.58,202.88,202.88,10274.58
1375,2021-04-25,Orange,California,269633,4933.00,6059,CA,County,3175692,8490.53,155.34,155.34,8490.53
1831,2021-04-25,Maricopa,Arizona,534430,9836.00,4013,AZ,County,4485414,11914.84,219.29,219.29,11914.84
2287,2021-04-25,Los Angeles,California,1232195,23802.00,6037,CA,County,10039107,12273.95,237.09,237.09,12273.95
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1202285,2021-04-25,Wheeler,Oregon,26,1.00,41069,OR,County,1332,1951.95,75.08,75.08,1951.95
1202480,2021-04-25,King,Texas,11,0.00,48269,TX,County,272,4044.12,0.00,0.00,4044.12
1202644,2021-04-25,Esmeralda,Nevada,38,0.00,32009,NV,County,873,4352.81,0.00,0.00,4352.81
1202804,2021-04-25,Loving,Texas,1,0.00,48301,TX,County,169,591.72,0.00,0.00,591.72


In [9]:
# Create ranking of infection rates by county for each state

counties_us.sort_values(by=['state', 'crate', 'county'], inplace=True, ascending=(True, False, True))
counties_us['state_case']=counties_us.groupby([counties_us.State2]).cumcount()+1
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_case
774579,2021-04-25,Hale,Alabama,2201,75.00,1065,AL,County,14651,15022.87,511.91,511.91,15022.87,1
843099,2021-04-25,Clarke,Alabama,3497,60.00,1025,AL,County,23622,14804.00,254.00,254.00,14804.00,2
606081,2021-04-25,Lowndes,Alabama,1382,53.00,1085,AL,County,9726,14209.34,544.93,544.93,14209.34,3
450966,2021-04-25,Franklin,Alabama,4221,82.00,1059,AL,County,31362,13458.96,261.46,261.46,13458.96,4
537406,2021-04-25,Etowah,Alabama,13737,352.00,1055,AL,County,102268,13432.35,344.19,344.19,13432.35,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1140639,2021-04-25,Platte,Wyoming,619,11.00,56031,WY,County,8393,7375.19,131.06,131.06,7375.19,19
739211,2021-04-25,Converse,Wyoming,990,17.00,56009,WY,County,13822,7162.49,122.99,122.99,7162.49,20
918956,2021-04-25,Lincoln,Wyoming,1389,12.00,56023,WY,County,19830,7004.54,60.51,60.51,7004.54,21
1022722,2021-04-25,Niobrara,Wyoming,151,2.00,56027,WY,County,2356,6409.17,84.89,84.89,6409.17,22


In [10]:
# Create ranking of death rates by county for each state

counties_us.sort_values(by=['state', 'drate', 'county'], inplace=True, ascending=(True, False, True))
counties_us['state_death']=counties_us.groupby([counties_us.State2]).cumcount()+1
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_case,state_death
606081,2021-04-25,Lowndes,Alabama,1382,53.00,1085,AL,County,9726,14209.34,544.93,544.93,14209.34,3,1
774579,2021-04-25,Hale,Alabama,2201,75.00,1065,AL,County,14651,15022.87,511.91,511.91,15022.87,1,2
257805,2021-04-25,Walker,Alabama,7175,278.00,1127,AL,County,63521,11295.48,437.65,437.65,11295.48,28,3
535818,2021-04-25,Clay,Alabama,1518,56.00,1027,AL,County,13235,11469.59,423.12,423.12,11469.59,23,4
653598,2021-04-25,Greene,Alabama,920,34.00,1063,AL,County,8111,11342.62,419.18,419.18,11342.62,26,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
652017,2021-04-25,Hot Springs,Wyoming,366,3.00,56017,WY,County,4413,8293.68,67.98,67.98,8293.68,17,19
918956,2021-04-25,Lincoln,Wyoming,1389,12.00,56023,WY,County,19830,7004.54,60.51,60.51,7004.54,21,20
896064,2021-04-25,Uinta,Wyoming,2158,12.00,56041,WY,County,20226,10669.44,59.33,59.33,10669.44,5,21
256596,2021-04-25,Teton,Wyoming,3719,9.00,56039,WY,County,23464,15849.81,38.36,38.36,15849.81,1,22


In [11]:
# Merge data from today with historical data to provide current rates + rates from previous points in time.

usa = pd.merge(counties, counties_us[['state', 'county','drate', 'crate', 'state_death', 'state_case']], on=['state', 'county'])
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case
0,2020-01-21,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,68.48,4273.53,19,23
1,2020-01-22,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,68.48,4273.53,19,23
2,2020-01-23,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,68.48,4273.53,19,23
3,2020-01-24,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,68.48,4273.53,19,23
4,2020-01-25,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,68.48,4273.53,19,23
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1200116,2021-04-21,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,4
1200117,2021-04-22,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,4
1200118,2021-04-23,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,4
1200119,2021-04-24,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,4


In [12]:
# Break out daily case data from the New York Times csv that is cumulative in nature

usa.sort_values(by=['state', 'state_case', 'date'], inplace=True)
usa['case_day'] = usa.groupby(['state','county']).cases.diff().fillna(usa.cases)
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day
771367,2020-03-30,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,511.91,15022.87,2,1,1.00
771368,2020-03-31,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,511.91,15022.87,2,1,0.00
771369,2020-04-01,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,511.91,15022.87,2,1,0.00
771370,2020-04-02,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,511.91,15022.87,2,1,0.00
771371,2020-04-03,Hale,Alabama,3,0.00,1065,AL,County,14651,20.48,0.00,511.91,15022.87,2,1,2.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
934320,2021-04-21,Crook,Wyoming,428,12.00,56011,WY,County,7584,5643.46,158.23,158.23,5643.46,6,23,0.00
934321,2021-04-22,Crook,Wyoming,428,12.00,56011,WY,County,7584,5643.46,158.23,158.23,5643.46,6,23,0.00
934322,2021-04-23,Crook,Wyoming,428,12.00,56011,WY,County,7584,5643.46,158.23,158.23,5643.46,6,23,0.00
934323,2021-04-24,Crook,Wyoming,428,12.00,56011,WY,County,7584,5643.46,158.23,158.23,5643.46,6,23,0.00


In [13]:
# Break out daily death data from the New York Times csv that is cumulative in nature

usa.sort_values(by=['state', 'state_death', 'date'], inplace=True)
usa['death_day'] = usa.groupby(['state','county']).deaths.diff().fillna(usa.deaths)
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
603653,2020-03-26,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,544.93,14209.34,1,3,1.00,0.00
603654,2020-03-27,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,544.93,14209.34,1,3,0.00,0.00
603655,2020-03-28,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,544.93,14209.34,1,3,0.00,0.00
603656,2020-03-29,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,544.93,14209.34,1,3,0.00,0.00
603657,2020-03-30,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,544.93,14209.34,1,3,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
601668,2021-04-21,Albany,Wyoming,4155,11.00,56001,WY,County,38880,10686.73,28.29,28.29,10761.32,23,4,7.00,0.00
601669,2021-04-22,Albany,Wyoming,4170,11.00,56001,WY,County,38880,10725.31,28.29,28.29,10761.32,23,4,15.00,0.00
601670,2021-04-23,Albany,Wyoming,4184,11.00,56001,WY,County,38880,10761.32,28.29,28.29,10761.32,23,4,14.00,0.00
601671,2021-04-24,Albany,Wyoming,4184,11.00,56001,WY,County,38880,10761.32,28.29,28.29,10761.32,23,4,0.00,0.00


In [14]:
# Export updated national data to csv for use in Tableau

usa.to_csv('daily_covid_us.csv', index=False)

In [15]:
# Filter California data to show only numbers for today

counties_cal = usa[(usa['date'] == '2021-04-25') & (usa['state'] == 'California')]
counties_cal

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
307745,2021-04-25,Imperial,California,28124,719.00,6025,CA,County,181215,15519.69,396.77,396.77,15519.69,1,2,16.00,0.00
2287,2021-04-25,Los Angeles,California,1232195,23802.00,6037,CA,County,10039107,12273.95,237.09,237.09,12273.95,2,5,369.00,17.00
153257,2021-04-25,San Bernardino,California,295510,4568.00,6071,CA,County,2180085,13554.98,209.53,209.53,13554.98,3,4,58.00,0.00
538946,2021-04-25,Inyo,California,1396,36.00,6027,CA,County,18039,7738.79,199.57,199.57,7738.79,4,25,1.00,0.00
68541,2021-04-25,Stanislaus,California,60753,1036.00,6099,CA,County,550660,11032.76,188.14,188.14,11032.76,5,9,0.00,0.00
31767,2021-04-25,Riverside,California,297881,4559.00,6065,CA,County,2470546,12057.29,184.53,184.53,12057.29,6,6,0.00,0.00
84978,2021-04-25,Tulare,California,49493,832.00,6107,CA,County,466195,10616.37,178.47,178.47,10616.37,7,10,0.00,0.00
54535,2021-04-25,San Joaquin,California,71972,1340.00,6077,CA,County,762148,9443.31,175.82,175.82,9443.31,8,19,0.00,0.00
30937,2021-04-25,Fresno,California,100991,1662.00,6019,CA,County,999101,10108.19,166.35,166.35,10108.19,9,14,63.00,0.00
488417,2021-04-25,Merced,California,31400,454.00,6047,CA,County,277680,11307.98,163.50,163.50,11307.98,10,8,0.00,0.00


In [16]:
# Show top 10 California counties based on largest current infection rate

counties_cal.sort_values(by=['crate'], inplace=True, ascending=False)
top10_ca_case_rates = counties_cal.head(10)
top10_ca_case_rates

<ipython-input-16-94608615179b>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_cal.sort_values(by=['crate'], inplace=True, ascending=False)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
1140190,2021-04-25,Lassen,California,5685,1.00,6035,CA,County,30573,18594.84,3.27,3.27,18594.84,56,1,0.00,0.00
307745,2021-04-25,Imperial,California,28124,719.00,6025,CA,County,181215,15519.69,396.77,396.77,15519.69,1,2,16.00,0.00
653540,2021-04-25,Kings,California,22841,246.00,6031,CA,County,152940,14934.61,160.85,160.85,14934.61,11,3,6.00,0.00
153257,2021-04-25,San Bernardino,California,295510,4568.00,6071,CA,County,2180085,13554.98,209.53,209.53,13554.98,3,4,58.00,0.00
2287,2021-04-25,Los Angeles,California,1232195,23802.00,6037,CA,County,10039107,12273.95,237.09,237.09,12273.95,2,5,369.00,17.00
31767,2021-04-25,Riverside,California,297881,4559.00,6065,CA,County,2470546,12057.29,184.53,184.53,12057.29,6,6,0.00,0.00
192681,2021-04-25,Kern,California,108280,1345.00,6029,CA,County,900202,12028.41,149.41,149.41,12028.41,14,7,24.00,0.00
488417,2021-04-25,Merced,California,31400,454.00,6047,CA,County,277680,11307.98,163.50,163.50,11307.98,10,8,0.00,0.00
68541,2021-04-25,Stanislaus,California,60753,1036.00,6099,CA,County,550660,11032.76,188.14,188.14,11032.76,5,9,0.00,0.00
84978,2021-04-25,Tulare,California,49493,832.00,6107,CA,County,466195,10616.37,178.47,178.47,10616.37,7,10,0.00,0.00


In [17]:
# Show top 10 California counties based on largest current death rate

counties_cal.sort_values(by=['drate'], inplace=True, ascending=False)
top10_ca_death_rates = counties_cal.head(10)
top10_ca_death_rates

<ipython-input-17-018d0b919d54>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_cal.sort_values(by=['drate'], inplace=True, ascending=False)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
307745,2021-04-25,Imperial,California,28124,719.00,6025,CA,County,181215,15519.69,396.77,396.77,15519.69,1,2,16.00,0.00
2287,2021-04-25,Los Angeles,California,1232195,23802.00,6037,CA,County,10039107,12273.95,237.09,237.09,12273.95,2,5,369.00,17.00
153257,2021-04-25,San Bernardino,California,295510,4568.00,6071,CA,County,2180085,13554.98,209.53,209.53,13554.98,3,4,58.00,0.00
538946,2021-04-25,Inyo,California,1396,36.00,6027,CA,County,18039,7738.79,199.57,199.57,7738.79,4,25,1.00,0.00
68541,2021-04-25,Stanislaus,California,60753,1036.00,6099,CA,County,550660,11032.76,188.14,188.14,11032.76,5,9,0.00,0.00
31767,2021-04-25,Riverside,California,297881,4559.00,6065,CA,County,2470546,12057.29,184.53,184.53,12057.29,6,6,0.00,0.00
84978,2021-04-25,Tulare,California,49493,832.00,6107,CA,County,466195,10616.37,178.47,178.47,10616.37,7,10,0.00,0.00
54535,2021-04-25,San Joaquin,California,71972,1340.00,6077,CA,County,762148,9443.31,175.82,175.82,9443.31,8,19,0.00,0.00
30937,2021-04-25,Fresno,California,100991,1662.00,6019,CA,County,999101,10108.19,166.35,166.35,10108.19,9,14,63.00,0.00
488417,2021-04-25,Merced,California,31400,454.00,6047,CA,County,277680,11307.98,163.50,163.50,11307.98,10,8,0.00,0.00


In [18]:
# Create dataframe with historical data for Top 10 counties based on current case rate

top10c_df = top10_ca_case_rates[['county', 'state']]
top10_ca_case_df = pd.merge(usa, top10c_df, on=['county', 'state'])
top10_ca_case_df['date'] = pd.to_datetime(top10_ca_case_df['date']).dt.strftime('%m/%d/%y')
top10_ca_case_df = top10_ca_case_df[['date', 'county','cases', 'deaths', 'current_crate', 'current_drate', 'drate', 'crate', 'state_death', 'state_case', 'case_day', 'death_day']]
top10_ca_case_df

,date,county,cases,deaths,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
0,03/20/20,Imperial,4,0.00,2.21,0.00,396.77,15519.69,1,2,4.00,0.00
1,03/21/20,Imperial,4,0.00,2.21,0.00,396.77,15519.69,1,2,0.00,0.00
2,03/22/20,Imperial,4,0.00,2.21,0.00,396.77,15519.69,1,2,0.00,0.00
3,03/23/20,Imperial,4,0.00,2.21,0.00,396.77,15519.69,1,2,0.00,0.00
4,03/24/20,Imperial,9,0.00,4.97,0.00,396.77,15519.69,1,2,5.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...
4032,04/21/21,Lassen,5679,1.00,18575.21,3.27,3.27,18594.84,56,1,6.00,0.00
4033,04/22/21,Lassen,5679,1.00,18575.21,3.27,3.27,18594.84,56,1,0.00,0.00
4034,04/23/21,Lassen,5685,1.00,18594.84,3.27,3.27,18594.84,56,1,6.00,0.00
4035,04/24/21,Lassen,5685,1.00,18594.84,3.27,3.27,18594.84,56,1,0.00,0.00


In [19]:
# Create dataframe with historical data for Top 10 counties based on current death rate

top10d_df = top10_ca_death_rates[['county', 'state']]
top10_ca_death_df = pd.merge(usa, top10d_df, on=['county', 'state'])
top10_ca_death_df['date'] = pd.to_datetime(top10_ca_death_df['date']).dt.strftime('%m/%d/%y')
top10_ca_death_df = top10_ca_death_df[['date', 'county','cases', 'deaths', 'current_crate', 'current_drate', 'drate', 'crate', 'state_death', 'state_case', 'case_day', 'death_day']]
top10_ca_death_df

,date,county,cases,deaths,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
0,03/20/20,Imperial,4,0.00,2.21,0.00,396.77,15519.69,1,2,4.00,0.00
1,03/21/20,Imperial,4,0.00,2.21,0.00,396.77,15519.69,1,2,0.00,0.00
2,03/22/20,Imperial,4,0.00,2.21,0.00,396.77,15519.69,1,2,0.00,0.00
3,03/23/20,Imperial,4,0.00,2.21,0.00,396.77,15519.69,1,2,0.00,0.00
4,03/24/20,Imperial,9,0.00,4.97,0.00,396.77,15519.69,1,2,5.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...
4118,04/21/21,Merced,31321,454.00,11279.53,163.50,163.50,11307.98,10,8,32.00,1.00
4119,04/22/21,Merced,31358,454.00,11292.86,163.50,163.50,11307.98,10,8,37.00,0.00
4120,04/23/21,Merced,31400,454.00,11307.98,163.50,163.50,11307.98,10,8,42.00,0.00
4121,04/24/21,Merced,31400,454.00,11307.98,163.50,163.50,11307.98,10,8,0.00,0.00


In [20]:
# Export California Top 10 case data to csv

top10_ca_case_df.to_csv('top10cases.csv', float_format='%.2f', index=False)

In [21]:
# Export California Top 10 death data to csv

top10_ca_death_df.to_csv('top10deaths.csv', float_format='%.2f', index=False)